In [1]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import pytorch3d as p3d
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pytorch3d.structures import Meshes
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.io import load_obj, load_ply, save_ply
from pytorch3d.loss import chamfer_distance

In [2]:
### config

test_date = "10-04-0754"
note_msg = """
这次的训练用的是3-64-256-512-256-3网络，加上了bn层，设置学习率变动为每30轮减半。
这次训练中展示的loss已经是平均的loss  
           """
# 定义点云数据的输入维度和输出维度
input_dim = 3  # 每个点的特征维度
output_dim = 3  # 输出点云的特征维度（可以与输入维度相同）

sxxxxes = os.listdir("../../data/all_results/")
train_sxxxxes = sxxxxes[:372]
test_sxxxxes = sxxxxes[372:]

device = "cuda" if torch.cuda.is_available() else "cpu"
num_epochs = 200
batch_size = 1


In [3]:
# 创建一个简单的全连接神经网络类
class PointCloudFCNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PointCloudFCNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)  # 输入层到隐藏层1
        self.bn1 = nn.BatchNorm1d(64)  # 添加BN层
        self.fc2 = nn.Linear(64, 256)  # 隐藏层1到隐藏层2
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 512)  # 隐藏层2到隐藏层3
        self.bn3 = nn.BatchNorm1d(512)
        self.fc4 = nn.Linear(512, 256)  # 隐藏层3到隐藏层4
        self.bn4 = nn.BatchNorm1d(256)
        self.fc5 = nn.Linear(256, output_dim)  # 隐藏层4到输出层

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # 在全连接层后添加BN
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.fc5(x)  # 输出层不使用激活函数
        return x


# DataSet类
class TrainingSet(Dataset):
    def __init__(self, s_name_list, transform=None, sample_num=5000) -> None:
        self.input = [
            f"../../data/all_results/{sname}/0_sphere.obj" for sname in s_name_list
        ]
        self.target = [
            f"../../data/all_results/{sname}/{sname}.ply" for sname in s_name_list
        ]
        self.sample_num = sample_num
        self.transform = transform

    def __len__(self):
        return len(self.input)

    def __getitem__(self, index):
        #对input采样5000个点
        input_tensor_points, input_tensor_faces, _ = load_obj(self.input[index], load_textures=False)
        input_mesh = Meshes([input_tensor_points], [input_tensor_faces.verts_idx]) #注意这里有个.verts_idx
        input_tensor_points_sampled = sample_points_from_meshes(input_mesh,5000).squeeze(0)
        #对output不使用采样
        target_tensor_points, target_tensor_faces = load_ply(self.target[index])
        target_mesh = Meshes([target_tensor_points],[target_tensor_faces])
        target_tensor_points_sampled = sample_points_from_meshes(target_mesh).squeeze(0)

        return input_tensor_points_sampled.to(device),target_tensor_points_sampled.to(device)

    def get_target(self, input):
        raise NotImplemented

In [4]:
#################################################################################
# pts,faces = load_ply(f"../../data/all_results/s0004_pulmonary_artery.nii.g_1/s0004_pulmonary_artery.nii.g_1.ply")
# mymesh = Meshes([pts],[faces])
# a = sample_points_from_meshes(mymesh,5)
# a = a.squeeze(0)
# a
# mymesh
#################################################################################

In [5]:
def plot_pointcloud(points, title=""):
    """Sample points uniformly from the surface of the mesh."""
    x, y, z = points.clone().detach().cpu().squeeze().unbind(1)
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter3D(x, z, -y)
    ax.set_xlabel("x")
    ax.set_ylabel("z")
    ax.set_zlabel("y")
    ax.set_title(title)
    ax.view_init(190, 30)
    plt.show()

In [6]:
my_train_dataset = TrainingSet(train_sxxxxes)
my_test_dataset = TrainingSet(test_sxxxxes)
train_data_loader = DataLoader(my_train_dataset, batch_size=batch_size, shuffle=False)


In [7]:
# 创建模型实例
model = PointCloudFCNet(input_dim, output_dim).to(device)
model

PointCloudFCNet(
  (fc1): Linear(in_features=3, out_features=64, bias=True)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=64, out_features=256, bias=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=256, out_features=512, bias=True)
  (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (bn4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc5): Linear(in_features=256, out_features=3, bias=True)
)

In [8]:
# 定义损失函数和优化器
criterion = chamfer_distance
optimizer = optim.Adam(model.parameters(), lr=0.01)  # 使用Adam优化器

# 创建一个StepLR学习率调度器，每个epoch后将学习率乘以0.9（这只是示例，你可以根据需要自定义）
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

In [9]:
# next(iter(data_loader))

In [10]:
# 打开log文件准备写入训练结果
logf = open(f"./logs/{test_date}.txt","w")

# 训练模型
for epoch in range(num_epochs):
    sum_loss = 0.0
    for input_point_cloud,target_point_cloud in train_data_loader:
        optimizer.zero_grad()
        output_point_cloud = model(input_point_cloud.squeeze(0))  # 前向传播 ，把(1,5000,3)变成了(5000,3) ,否则bn层会报错
        loss,_ = criterion(output_point_cloud.unsqueeze(0), target_point_cloud)  # 计算损失 ，重新把(5000,3)变回(1,5000,3)
        sum_loss += loss.item()
        loss.backward()  # 反向传播
        optimizer.step()  # 优化模型参数

    current_lr = optimizer.param_groups[0]['lr']
    lr_scheduler.step()

    print(f'[{time.asctime()}]   Epoch [{epoch+1}/{num_epochs}],current_lr ={current_lr}, Loss: {sum_loss/len(my_train_dataset):.6f}',file=logf)
    logf.flush()

    # if (epoch + 1) % 5 == 0:
    #     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        # plot_pointcloud(output_point_cloud)# 绘制点云


logf.close()

In [11]:
# 保存模型

torch.save(model,f"./models/{test_date}.pt")

In [12]:
# 加载模型

model = torch.load(f"./models/{test_date}.pt")

In [13]:
# 使用训练好的模型生成新的点云数据
try_input_point_cloud,_,_ = load_obj("../../data/all_results/s1272_pulmonary_artery.nii.g_1/0_sphere.obj", load_textures=False)
new_point_cloud = model(try_input_point_cloud.to(device))
new_point_cloud
# 输出的new_point_cloud包含了经过神经网络处理后的新点云数据

tensor([[ 0.1145,  0.0044,  0.2180],
        [ 0.5254, -0.1371,  0.1565],
        [ 0.4680, -0.1040,  0.1493],
        ...,
        [ 0.0727,  0.0514, -0.0985],
        [ 0.1013,  0.0431, -0.0980],
        [-0.1123,  0.1447, -0.1163]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [14]:
# 计算对new_point_cloud的loss
torch.cuda.empty_cache()
test_data_loader = DataLoader(my_test_dataset, batch_size=1, shuffle=False)
sum_loss = 0.0
for input_point_cloud_test,target_point_cloud_test in test_data_loader:
    # save_ply("./outTTT.ply",model(x))
    # save_ply("./tarTTT.ply",target_point_cloud_test)
    # save_ply("./inTTT.ply",x)
    lossT,_ = criterion(model(input_point_cloud_test.squeeze(0)).unsqueeze(0), target_point_cloud_test)
    sum_loss += lossT.item()

with open(f"./logs/{test_date}.txt","a") as f:
    print(f"\n\navg_loss on test_set: {sum_loss/len(my_test_dataset):.6f}",file=f)
    print(f"\n\n{note_msg}",file=f)



In [15]:
#看一眼新的输出长啥样
save_ply(f"./plys/s1272_{test_date}.ply",new_point_cloud)